# 기존 프로그램의 단점

기존에 작성한 다운로드 기능에서는 다운로드가 되고 있는지 현재 상황을 알 수가 없고 오직 다운로드가 완료되었을때만 상태를 확인할 수 있습니다. 이를 극복하기 위해선 다운로드가 되고 있는 상황 중간중간의 상태를 체크하고 확인 할 수 있어야 하는데 이를 구현하기 위해서는 몇가지 추가 작업을 해야 합니다.


### 웹소켓

웹 환경에서 실시간 양방향 통신을 하기 위해서는 웹소켓 기능을 사용해야 합니다. 웹소켓은 HTML5 의 새로운 기능으로 웹서버와 웹브라우저가 지속적으로 실시간 데이터를 주고 받을 수 있게 해줍니다. 이 웹소켓을 활용하여 기존에 작성한 페이지에 각각 동영상이 다운로드 되는 프로그래스바를 구현해보도록 하겠습니다.


### 웹에서의 웹소켓  [[웹소켓 CDN 공식링크]](https://cdnjs.com/libraries/socket.io)

html 에서의 웹소켓은 socket.io 라는 라이브러를 통해서 사용할 수 있습니다.

In [ ]:
%%html

<script type="text/javascript" src="//cdnjs.cloudflare.com/ajax/libs/socket.io/2.2.0/socket.io.js"></script>

웹소켓을 사용하기 위해선 html 파일 상단에 websocket 라이브러리를 사용하기 위한 링크를 작성해야 합니다.


### 파이썬 플라스크에서의 웹소켓 [[Flask-SocketIO 공식링크]](https://flask-socketio.readthedocs.io/en/latest/)

파이썬에서는 flask_socketio 라이브러리를 먼저 설치 해야 합니다. 

``` pip install flask-socketio```

In [ ]:
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.secret_key = "testkey"   # 임의의 비밀키값
websocket = SocketIO(app, async_handlers=True)


if __name__ == "__main__":
    websocket.run(app, host="0.0.0.0", port=9900, debug=True)

라이브러리 사용을 위해 먼저 flask_socketio 라이브러리에 있는 SocketIO 와 클라이언트 전송을 위한 emit 을 import 해야 합니다. 그리고 app 로 선언된 플라스크 인스턴스를 SocketIO 로 래핑하여 websocket 이란 인스턴스를 생성합니다. 앞으로 플라스크의 모든 동작은 websocket 변수를 사용하게 됩니다. 따라서 ```app.run()``` 으로 시작하던 코드 역시 ```websocket.run()``` 으로 수정해야 합니다.

SocketIO 생성자에 async_handlers 인자값을 True 로 설정해야 소켓처리가 개별쓰레드로 동작하게 됩니다. 

### youtube_dl 라이브러 기능 추가 [[공식링크]](https://github.com/ytdl-org/youtube-dl/)

youtube_dl 라이브러리에는 다운로드 상황을 확인할 수 있게 Hook을 걸어주는 기능을 제공합니다. 보통 Hook 이란 쉽게 예를 들어 A 함수가 B 로 결과를 리턴한다고 가정했을때 A 함수에 훅을 걸어 B 로 리턴하기전에 결과를 가로채는 기능이라고 볼 수 있습니다. 여기서는 훅을 걸어서 다운로드 중간 중간 외부에 있는 함수를 호출해주는 콜백함수 같이 동작을 하게 되며 최초 옵션에 정의하여 사용할 수 있습니다.

In [ ]:
def download_hook(d):
    if d['status'] == 'finished':
        file_tuple = os.path.split(os.path.abspath(d['filename']))
        print("<<<<<<< Done downloading {} >>>>>>>".format(file_tuple[1]))
    if d['status'] == 'downloading':
        print("<<<<<< {}, {}, {} >>>>>".format(d['filename'], d['_percent_str'], d['_eta_str']))


def download_from_youtube(vid):
    url = "https://www.youtube.com/watch?v={}".format(vid)

    ydl_opts = {
        'format': 'best/best',      # 가장 좋은 화질로 선택(화질을 선택하여 다운로드 가능)
        'writesubtitles': 'best',   # 자막 다운로드(자막이 없는 경우 다운로드 X)
        'writethumbnail': 'best',   # 영상 thumbnail 다운로드
        'writeautomaticsub': True,  # 자동 생성된 자막 다운로드
        'subtitleslangs': 'kr',     # 자막 언어가 영어인 경우(다른 언어로 변경 가능)
        'progress_hooks': [download_hook],
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info_dict)
            print("다운로드 성공 {}".format(filename))
    except Exception as e:
        print('error', e)

```def download_hook(d)``` 이라는 함수를 작성하고 인자로 넘어온 d 에 다운로드의 상태를 확인할 수 있는 여러가지 정보가 담겨 있습니다. 이 ```download_hook``` 함수는 ```ydl_opts``` 옵션에 ```progress_hooks``` 라는 키에 넘겨주어야 하는데 이때 리스트 형태로 넘겨주어야 합니다.

### 문제점

여기서 우리는 어떤 요청이 다운로드 되었는지를 알아야 하는데 그러기 위해서는 ```vid``` 값을 Hook 으로 동작하는 ```download_hook(d)``` 함수에 넘겨줄 수 있어야 하는데 ```download_hook(d)``` 함수는 기본적으로 넘어오는 d 의 인자에 설정된 요소들 외에 우리가 임의로 인자를 넘길 수 있는 방법이 없습니다. 그래서 우리는 좀 다른 방법으로 download_hook 함수를 download_from_youtube 함수 안에 inner 함수로 등록하여 사용해서 이 문제를 해결할 수 있습니다.

In [ ]:
def download_from_youtube(vid):
    url = "https://www.youtube.com/watch?v={}".format(vid)

    def inner_download_hook(d):
        if d['status'] == 'finished':
            file_tuple = os.path.split(os.path.abspath(d['filename']))
            print("<<<<<<< Done downloading {} >>>>>>>".format(file_tuple[1]))
            emit("complete", {"vid": vid})
        if d['status'] == 'downloading':
            print("<<<<<< {}, {}, {} >>>>>".format(d['filename'], d['_percent_str'], d['_eta_str']))
            per = d['_percent_str']
            emit("status", {"vid": vid, "per": per})

    ydl_opts = {
        'format': 'best/best',      # 가장 좋은 화질로 선택(화질을 선택하여 다운로드 가능)
        'writesubtitles': 'best',   # 자막 다운로드(자막이 없는 경우 다운로드 X)
        'writethumbnail': 'best',   # 영상 thumbnail 다운로드
        'writeautomaticsub': True,  # 자동 생성된 자막 다운로드
        'subtitleslangs': 'kr',     # 자막 언어가 영어인 경우(다른 언어로 변경 가능)
        'progress_hooks': [inner_download_hook],
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info_dict)
            print("다운로드 성공 {}".format(filename))
    except Exception as e:
        print('error', e)

In [ ]:
%%html

<script src="https://code.jquery.com/jquery-3.4.1.min.js"></script>
<script type="text/javascript" src="//cdnjs.cloudflare.com/ajax/libs/socket.io/2.2.0/socket.io.js"></script>
<script>
    $(document).ready(function() {
        var socket = io.connect("http://" + document.domain + ":" + location.port);
        socket.on("connect", function() {
            console.log("server socket connected");
        });
</script>

html 페이지에서 html 페이지가 ```$(document).ready()```(로드완료) 되면 현재 접속된 페이지의 도메인과 포트로 접속하여 ```var socket``` 이라는 변수에 객체를 저장합니다. 이 ```socket``` 이라는 변수에 저장된 객체를 통해 우리는 서버와 소켓 통신을 할 수 있게 됩니다.

```socket.on("connect", function() {}``` 은 서버에 접속되면 connect 이벤트가 발생하는데 이 이벤트를 처리하는 내용을 담고 있습니다. 여기서는 별다른 작업은 없고 그냥 브라우저 개발자모드 콘솔창에 문자열만 출력하도록 하였습니다.

In [ ]:
%%html

{% if datas|length > 0 %}
    <table>
    {% for d in datas %}
        <tr>
            <td><input type="checkbox" name="vids" value="{{d.vid}}"></td>
            <td><img src="{{d.thumb}}" width="100"></td>
            <td>{{d.title}}</td>
            <td><a href="{{d.url}}">{{d.url}}</a></td>
        </tr>
        <tr>
            <td colspan="4"><progress id="p_{{d.vid}}" value="0" max="100" style="width:100%"></progress></td>
        </tr>
    {% endfor %}
    </table>
    <a id="download" href="#">다운로드</a>
{% endif %}

기존의 html 코드에 progressbar 를 추가하는 코드를 작성하고 이 프로그래스바의 이름은 동영상의 vid 앞에 ```p_``` 를 붙이도록 했습니다. 여기서 어떤 동영상이 얼마나 다운로드 중이다를 표현하기 위해선 이 프로그래스바로 접근을 할 수 있어야 하기에 이름이 중요합니다. 
다운로드 링크의 ```href``` 는 더이상 사용하지 않고 # 으로 수정하고 이벤트로 대신 하기 위해 아이디를 downlaod 라고 했습니다.

In [ ]:
%%html

$("#download").on("click", function() {
        console.log("download click");
        var vids = []
        // name이 vids 인 checkbox에서 checked 된 요소를 반복한다.
        $("input[name=vids]:checked").each(function() {
            var vid = $(this).val();
            vids.push(vid)
        });
        for(var i = 0 ; i < vids.length ; i++) {
            vid = vids[i];
            socket.emit("download", vid);
        }
    })

기존의 다운로드 함수를 위의 코드처럼 이벤트로 대체 합니다. ```(#download).on("click", function(){}) ``` 는 downlaod 라는 이름의 요소가 on("click") 클릭되면 호출되게 됩니다. for 루프문 안에서는 해당 vid 값을 서버에 다운로드 요청을 합니다.
socket 은 기본적으로 send() 함수와 emit() 함수를 제공하는데 emit() 함수는 사용자가 지정한 이벤트를 요청하는 기능을 수행할 수 있습니다.

```emit("이벤트", "데이터");```

```send("데이터")```

여기서 우리는 "download" 라는 이름의 이벤트에 vid 값을 서버에 요청하였습니다.

In [ ]:
@websocket.on("download")
def socket_messgae(vid):
    download_from_youtube(vid)
    emit("complete", {"vid": vid})

파이썬 플라스크에서는 ``` @websocket.on``` 데코레이터를 정의하여 이벤트를 처리 할 수 있습니다. 클라이언트에서 요청한 download 라는 이벤트를 위와 같이 정의하고 넘어온 vid 값을 위에서 수정한 ```download_from_youtube``` 함수로 넘겨줍니다.

In [ ]:
def inner_download_hook(d):
    if d['status'] == 'finished':
        file_tuple = os.path.split(os.path.abspath(d['filename']))
        print("<<<<<<< Done downloading {} >>>>>>>".format(file_tuple[1]))
        emit("complete", {"vid": vid})
    if d['status'] == 'downloading':
        print("<<<<<< {}, {}, {} >>>>>".format(d['filename'], d['_percent_str'], d['_eta_str']))
        per = d['_percent_str']
        emit("status", {"vid": vid, "per": per})

위에서 수정한 download_from_youtube 함수 안에 정의된 inner_download_hook 함수에 보면 emit() 함수를 사용해서 클라이언트에 정보를 전송하는 내용이 있습니다. 여기서 우리는 동영상 다운로드가 완료된 complete 라는 이벤트와 다운로드 중일때의 status 라는 두가지의 이벤트를 클라이언트에 전송하게 됩니다. 다운로드가 완료된 complete 이벤트에는 다운로드 완료된 동영상의 vid 값을 전송하고 다운로드 중일때는 vid 값과 현재 다운로드 퍼센트를 전송합니다.

In [ ]:
%%html

socket.on("status", function(data) {
        console.log(data);
        vid = data.vid;
        per = data.per;
        console.log(vid);
        element = "p_" + vid;
        $("#" + element).val(parseInt(per));
    });
    socket.on("complete", function(data) {
        console.log(data);
        vid = data.vid;
        console.log(vid);
        element = "p_" + vid;
        $("#" + element).val(100);
    });

html에는 서버에서 전송된 status 와 complete 에 해당하는 이벤트를 처리 하기 위한 함수를 작성해야 합니다. 서버에서 넘어온 vid 값 앞에 p_ 를 조합하여 프로그래바의 id 를 작성하고 해당 프로그래스바의 val 값을 설정하여 진행상태를 표시할 수 있습니다.

![download.gif](images/download.gif)

In [ ]:
from flask import Flask
from flask import render_template
from flask import request
from flask_socketio import SocketIO, emit
import youtube_dl
import requests
import re
import os

app = Flask(__name__)
app.secret_key = "testkey"
websocket = SocketIO(app, async_handlers=True)
# websocket = SocketIO(app, pingTimeout=900)


def download_from_youtube(vid):
    url = "https://www.youtube.com/watch?v={}".format(vid)

    def inner_download_hook(d):
        if d['status'] == 'finished':
            file_tuple = os.path.split(os.path.abspath(d['filename']))
            print("<<<<<<< Done downloading {} >>>>>>>".format(file_tuple[1]))
            emit("complete", {"vid": vid})
        if d['status'] == 'downloading':
            print("<<<<<< {}, {}, {} >>>>>".format(d['filename'], d['_percent_str'], d['_eta_str']))
            per = d['_percent_str']
            emit("status", {"vid": vid, "per": per})

    ydl_opts = {
        'format': 'best/best',      # 가장 좋은 화질로 선택(화질을 선택하여 다운로드 가능)
        'writesubtitles': 'best',   # 자막 다운로드(자막이 없는 경우 다운로드 X)
        'writethumbnail': 'best',   # 영상 thumbnail 다운로드
        'writeautomaticsub': True,  # 자동 생성된 자막 다운로드
        'subtitleslangs': 'kr',     # 자막 언어가 영어인 경우(다른 언어로 변경 가능)
        'progress_hooks': [inner_download_hook],
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            info_dict = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info_dict)
            print("다운로드 성공 {}".format(filename))
    except Exception as e:
        print('error', e)


def get_playlist_str(url):
    r = requests.get(url, headers={"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"})
    _parse = 'var ytInitialData = '
    _str = r.text[r.text.find(_parse) + len(_parse):r.text.find("\n", r.text.find(_parse) + len(_parse))-1]
    _list_str = _str.split('playlistVideoRenderer":{')

    lists = []
    for l in _list_str:
        _p1 = '"videoId":'
        _p2 = 'thumbnails":[{"url":'
        _p3 = '{"text":"'
        _p4 = '"lengthSeconds":"'
        _p5 = '"isPlayable":'
        vid = l[l.find(_p1) + len(_p1):l.find(",", l.find(_p1) + len(_p1))].replace('"', "")
        thumb = l[l.find(_p2) + len(_p2):l.find(",", l.find(_p2) + len(_p2))].replace('\u0026', '&').replace('"', "")
        title = l[l.find(_p3) + len(_p3):l.find('"}', l.find(_p3) + len(_p3))].replace('"}', "")
        l = l[l.find("lengthText"):]
        length = l[l.find(_p4) + len(_p4):l.find(',', l.find(_p4) + len(_p4))].replace('"', "")
        isplay = l[l.find(_p5) + len(_p5):l.find(',', l.find(_p5) + len(_p5))]

        pattern = re.compile(r'[0-9]+ ?[화,회,편]')
        result = pattern.search(title)
        seq = None
        if result is not None:
            seq = result.group()
        if not isplay:
            continue

        lists.append({
            "vid": vid,
            "url": "http://www.youtube.com/watch?v={}".format(vid),
            "thumb": thumb,
            "title": title,
            "sec": length,
            "isplay": isplay,
            "seq": seq,
        })
    return lists


@app.route("/", methods=["GET", "POST"])
def index():
    print(request.method)
    if request.method == "GET":
        return render_template("index_1.html")
    else:
        _url = request.form.get("url")
        if _url.find("/playlist?list=") < 0:
            return "Error"

        lists = get_playlist_str(_url)
        return render_template("index_3.html", datas=lists)


@websocket.on("download")
def socket_messgae(vid):
    download_from_youtube(vid)
    emit("complete", {"vid": vid})
    print(">>>> socket {}".format(vid))


if __name__ == "__main__":
    websocket.run(app, host="0.0.0.0", port=9900, debug=True)


In [ ]:
%%html

<script src="https://code.jquery.com/jquery-3.4.1.min.js"></script>
<script type="text/javascript" src="//cdnjs.cloudflare.com/ajax/libs/socket.io/2.2.0/socket.io.js"></script>
<script>
    $(document).ready(function() {
        var socket = io.connect("http://" + document.domain + ":" + location.port);
        socket.on("connect", function() {
            console.log("socket connected");
        });
        socket.on("status", function(data) {
            console.log(data);
            vid = data.vid;
            per = data.per;
            console.log(vid);
            element = "p_" + vid;
            $("#" + element).val(parseInt(per));
        });
        socket.on("complete", function(data) {
            console.log(data);
            vid = data.vid;
            console.log(vid);
            element = "p_" + vid;
            $("#" + element).val(100);
        });

        $("#download").on("click", function() {
            console.log("download click");
            var vids = []
            // name이 vids 인 checkbox에서 checked 된 요소를 반복한다.
            $("input[name=vids]:checked").each(function() {
                var vid = $(this).val();
                vids.push(vid)
            });
            for(var i = 0 ; i < vids.length ; i++) {
                vid = vids[i];
                socket.emit("download", vid);
            }
        });
    });
</script>

<form id="form" action="/" method="POST">
    Playlist 주소: <input type="text" name="url" id="url" value="">
    <input type="submit" value="체크">
</form>

{% if datas|length > 0 %}
    <table>
    {% for d in datas %}
        <tr>
            <td><input type="checkbox" name="vids" value="{{d.vid}}"></td>
            <td><img src="{{d.thumb}}" width="100"></td>
            <td>{{d.title}}</td>
            <td><a href="{{d.url}}">{{d.url}}</a></td>
        </tr>
        <tr>
            <td colspan="4"><progress id="p_{{d.vid}}" value="0" max="100" style="width:100%"></progress></td>
        </tr>
    {% endfor %}
    </table>
    <a id="download" href="#">다운로드</a>
{% endif %}